In [40]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


In [41]:
# Load data
fp = "../Resources/Fire_Data_Over_1.csv"

map_df = pd.read_csv(fp)

map_df.head()

,Unnamed: 0,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,13,14,TIGER,2004,2453276.5,272,1200.0,4.0,Campfire,6.0,B,38.433333,-120.510000,CA,5
1,16,17,POWER,2004,2453284.5,280,1415.0,2.0,Equipment Use,16823.0,G,38.523333,-120.211667,CA,5
2,17,18,FREDS,2004,2453291.5,287,1618.0,2.0,Equipment Use,7700.0,G,38.780000,-120.260000,CA,17
3,24,25,EAGLE,2004,2453187.5,183,1712.0,1.0,Lightning,2.5,B,33.545278,-105.229444,NM,27
4,25,26,BACHELOR,2004,2453206.5,202,1405.0,1.0,Lightning,10.0,C,33.315833,-105.512222,NM,27


In [42]:
# Remove Puerto Rico since it won't show up on our map

new = map_df.loc[(map_df.loc[:,'STATE']!='PR')]
new.head()

,Unnamed: 0,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,13,14,TIGER,2004,2453276.5,272,1200.0,4.0,Campfire,6.0,B,38.433333,-120.510000,CA,5
1,16,17,POWER,2004,2453284.5,280,1415.0,2.0,Equipment Use,16823.0,G,38.523333,-120.211667,CA,5
2,17,18,FREDS,2004,2453291.5,287,1618.0,2.0,Equipment Use,7700.0,G,38.780000,-120.260000,CA,17
3,24,25,EAGLE,2004,2453187.5,183,1712.0,1.0,Lightning,2.5,B,33.545278,-105.229444,NM,27
4,25,26,BACHELOR,2004,2453206.5,202,1405.0,1.0,Lightning,10.0,C,33.315833,-105.512222,NM,27


In [43]:
# Sort by year ascending 

df = new.sort_values(["FIRE_YEAR", "STATE"])
df.dropna(how="any")
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,53189,53637,NaN,1992,2448753.5,132,730.0,5.0,Debris Burning,2.0,B,60.500000,-149.866667,AK,NaN
1,53190,53638,NaN,1992,2448754.5,133,1301.0,5.0,Debris Burning,40.0,C,60.500000,-149.866667,AK,NaN
2,53193,53641,NaN,1992,2448772.5,151,2030.0,9.0,Miscellaneous,2.5,B,60.733333,-149.383333,AK,NaN
3,53197,53645,NaN,1992,2448777.5,156,1504.0,5.0,Debris Burning,2.0,B,60.183333,-149.333333,AK,NaN
4,173288,175112,NaN,1992,2448783.5,162,1830.0,12.0,Structure,27.0,C,61.849300,-156.352300,AK,NaN


In [44]:
# Simplify to two columns and count number of wildfires in each year

group_df = df[['FIRE_YEAR','STATE']]
state_totals = list(df.groupby("STATE")["FIRE_YEAR"].count())
year_totals = list(group_df["FIRE_YEAR"].value_counts())

group_df.head()

,FIRE_YEAR,STATE
0,1992,AK
1,1992,AK
2,1992,AK
3,1992,AK
4,1992,AK


In [106]:
# Make a unique data frame to pull unique years and states

unique_df = group_df.drop_duplicates().reset_index(drop=True)
unique_states = group_df.loc[group_df["FIRE_YEAR"] == 2015].drop_duplicates().reset_index(drop=True)

unique_df.head()

,FIRE_YEAR,STATE
0,1992,AK
1,1992,AL
2,1992,AR
3,1992,AZ
4,1992,CA


In [107]:
states = unique_states["STATE"]
years = list(range(1992,2016))

# Make a df of totals for the years

years_df = pd.DataFrame({
    "Year": years,
    "Totals": year_totals
})
years_df.head()

#could make a line graph with this data

,Year,Totals
0,1992,49583
1,1993,41500
2,1994,40036
3,1995,36389
4,1996,35920


In [108]:
# Make a df of totals for the states

states_df = pd.DataFrame({ 
    "State": states, 
    "Totals": state_totals
})
states_df.head()

#this data will be used later to make our cloropleth map of wildfires across all years 

,State,Totals
0,AK,4326
1,AL,44588
2,AR,21808
3,AZ,15342
4,CA,39563


In [109]:
# Get the number of wildfires in each year in each state

df = df[["FIRE_YEAR", "STATE"]]
totals = list(df.groupby(["FIRE_YEAR", "STATE"])["STATE"].count())
unique_df["Totals"] = totals
unique_df.head()

,FIRE_YEAR,STATE,Totals
0,1992,AK,133
1,1992,AL,58
2,1992,AR,89
3,1992,AZ,936
4,1992,CA,2299


In [110]:
# Use pd.pivot_tableto make a new dataframe 

unique_df = pd.pivot_table(unique_df, values = 'Totals', index=['STATE'], columns = 'FIRE_YEAR').reset_index()
unique_df.head()

FIRE_YEAR,STATE,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,AK,133.0,254.0,205.0,92.0,191.0,254.0,73.0,165.0,102.0,...,80.0,223.0,114.0,195.0,310.0,128.0,144.0,201.0,78.0,403.0
1,AL,58.0,96.0,2331.0,2715.0,2707.0,1643.0,2314.0,2849.0,4125.0,...,2980.0,2585.0,1374.0,1024.0,2114.0,1516.0,1052.0,996.0,1366.0,1183.0
2,AR,89.0,88.0,98.0,149.0,147.0,716.0,730.0,1457.0,1578.0,...,1883.0,935.0,583.0,829.0,1600.0,1672.0,1555.0,561.0,958.0,868.0
3,AZ,936.0,1733.0,1201.0,1430.0,720.0,347.0,547.0,518.0,689.0,...,750.0,387.0,341.0,560.0,284.0,460.0,412.0,335.0,291.0,323.0
4,CA,2299.0,2092.0,2038.0,1788.0,2398.0,1658.0,1470.0,1947.0,1399.0,...,1921.0,1674.0,1743.0,1247.0,1135.0,1597.0,1297.0,1489.0,938.0,1134.0


In [115]:
# Now to make cloropleth maps 

for col in states_df.columns:
    states_df[col] = states_df[col].astype(str)

scl = [[0.0, 'rgb(255,215,0)'],
       [0.2, 'rgb(255,165,0)'],
       [0.4, 'rgb(255,140,0)'],
       [0.6, 'rgb(255,115,0)'],
       [0.8, 'rgb(255,90,0)'],
       [1.0, 'rgb(255,65,0)']]

df['text'] = states_df['State']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = states_df['State'],
        z = unique_df[1996].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Number of Wildfires")
        ) ]

layout = dict(
        title = 'USA Wildfires 1996 </br></br>Source: Kaggle</br></br> Note: black indicates none',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [57]:
#Bonus: make a for loop to save each year's map as png
years = list(range(1992,2016))

import plotly.io as pio

static_image_bytes = pio.to_image(fig, format='png')

from IPython.display import Image

Image(static_image_bytes)

for year in years:
    pio.write_image(fig, file=f'uswildfire{year}.png', format='png')
    savefig("../maps/f'uswildfire{year}.png'", bbox_inches='tight')
    
